In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from typing import List
from langchain_core.documents import Document
import os
import logging
from dotenv import load_dotenv, find_dotenv
import sys
sys.path.append("/home/phucuy2025/RAG-Chatbot/api")
sys.path.append("/home/phucuy2025/RAG-Chatbot")
load_dotenv(find_dotenv(), override=True)
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

from api.vector_store_utils import vector_store
# retriever = vector_store.as_retriever(search_kwargs={"k": 2})
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.5},
)
output_parser = StrOutputParser()

logging.basicConfig(filename="rag_chatbot_app.log", level=logging.INFO)


# Set up prompts and chains
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# system_prompt = (
#     "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )

system_prompt = (
    "Bạn là trợ lý cho các nhiệm vụ trả lời câu hỏi. "
    "Sử dụng các phần ngữ cảnh sau đây để trả lời "
    "câu hỏi. Nếu bạn không biết câu trả lời, hãy nói rằng bạn "
    "không biết. Sử dụng tối đa ba câu và giữ cho câu trả lời ngắn gọn."
     "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)



def get_rag_chain(model="gemini-2.0-flash-001"):
    llm = ChatGoogleGenerativeAI(model=model, google_api_key=GOOGLE_API_KEY)
    history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return rag_chain

/home/phucuy2025/RAG-Chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import uuid
from api.new_db_util import (
    insert_application_logs, 
    get_chat_history, 
    get_all_documents, 
    insert_document_record, 
    delete_document_record,
    create_application_logs,
    create_document_store
)

session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)

rag_chain = get_rag_chain()

answer = rag_chain.invoke({
    "input": "Nói cho tôi biết về địa đạo Củ Chi",
    "chat_history": chat_history
})

answer

{'input': 'Nói cho tôi biết về địa đạo Củ Chi',
 'chat_history': [],
 'context': [],
 'answer': 'Địa đạo Củ Chi là một mạng lưới đường hầm kết nối nằm ở quận Củ Chi, Thành phố Hồ Chí Minh, Việt Nam. Các đường hầm được xây dựng trong Chiến tranh Việt Nam và từng là căn cứ hoạt động của Việt Cộng. Du khách có thể bò qua các đoạn đường hầm an toàn hơn.'}

In [9]:
import requests

def get_api_response(question, session_id, model):
    headers = {'accept': 'application/json', 'Content-Type': 'application/json'}
    data = {'question': question, 'model': model}
    if session_id:
        data['session_id'] = session_id
    
    try:
        response = requests.post("http://localhost:8000/chat", headers=headers, json=data)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"API request failed with status code {response.status_code}: {response.text}")
            return None
    except Exception as e:
        print(f"an error occurred: {str(e)}")
        return None
tmp = get_api_response("Nói cho tôi biết về địa đạo Củ Chi", None, "gemini-2.0-flash-001")
print(tmp)
print()
session_id = tmp.get('session_id')
print(session_id)
chat_history = get_chat_history(session_id)
chat_history

{'answer': 'Địa đạo Củ Chi là một hệ thống đường hầm rộng lớn ở Việt Nam, được sử dụng bởi các chiến binh Việt Cộng trong Chiến tranh Việt Nam. Các đường hầm được sử dụng làm nơi ẩn náu, thông tin liên lạc và tiếp tế, bệnh viện và khu sinh hoạt cho các chiến binh Việt Cộng. Ngày nay, địa đạo Củ Chi là một điểm thu hút khách du lịch nổi tiếng.', 'session_id': '2b192f3d-f63f-43c9-bf9b-831409a5e903', 'model': 'gemini-2.0-flash-001'}

2b192f3d-f63f-43c9-bf9b-831409a5e903


[]

703f2db1-6375-46d8-ae86-6b46a7f3fbbd


[]

In [1]:
from langchain_core.chat_history import (
    BaseChatMessageHisotry,
    InMemoryChatMessageHistory
)

ImportError: cannot import name 'BaseChatMessageHisotry' from 'langchain_core.chat_history' (/home/phucuy2025/RAG-Chatbot/venv/lib/python3.12/site-packages/langchain_core/chat_history.py)